In [ ]:
import re
import string
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
import tensorflow as tf

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter

In [ ]:
import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM,
                          Embedding,
                          BatchNormalization,
                          Dense,
                          TimeDistributed,
                          Dropout,
                          Bidirectional,
                          Flatten,
                          GlobalMaxPool1D)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

In [ ]:
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    accuracy_score
)

In [ ]:
df = pd.read_csv('dark_dataset.csv')

In [ ]:
df['text_len'] = df['text'].apply(lambda x: len(x.split(' ')))
df.head()

,text,label,text_len
0,Connect with Facebook for a personalized exper...,1,7
1,Oops! Something went wrong. Please try again l...,1,8
2,"By clicking 'Accept,' you agree to our terms a...",1,10
3,Great deals! Checkout now.,1,4
4,Click here to manage your email preferences.,1,7


In [ ]:
max(df['text_len'])

143

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
df['text_clean'] = df['text'].apply(clean_text)
df.head()

,text,label,text_len,text_clean
0,Connect with Facebook for a personalized exper...,1,7,connect with facebook for a personalized exper...
1,Oops! Something went wrong. Please try again l...,1,8,oops something went wrong please try again later
2,"By clicking 'Accept,' you agree to our terms a...",1,10,by clicking accept you agree to our terms and ...
3,Great deals! Checkout now.,1,4,great deals checkout now
4,Click here to manage your email preferences.,1,7,click here to manage your email preferences


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

df['text_clean'] = df['text_clean'].apply(remove_stopwords)
df.head()

,text,label,text_len,text_clean
0,Connect with Facebook for a personalized exper...,1,7,connect facebook personalized experience
1,Oops! Something went wrong. Please try again l...,1,8,oops something went wrong please try later
2,"By clicking 'Accept,' you agree to our terms a...",1,10,clicking accept agree terms conditions
3,Great deals! Checkout now.,1,4,great deals checkout
4,Click here to manage your email preferences.,1,7,click manage email preferences


In [ ]:
stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

In [ ]:
df['text_clean'] = df['text_clean'].apply(stemm_text)
df.head()

,text,label,text_len,text_clean
0,Connect with Facebook for a personalized exper...,1,7,connect facebook person experi
1,Oops! Something went wrong. Please try again l...,1,8,oop someth went wrong pleas tri later
2,"By clicking 'Accept,' you agree to our terms a...",1,10,click accept agre term condit
3,Great deals! Checkout now.,1,4,great deal checkout
4,Click here to manage your email preferences.,1,7,click manag email prefer


In [ ]:
def preprocess_data(text):
    text = clean_text(text)

    text = ' '.join(word for word in text.split(' ') if word not in stop_words)

    text = ' '.join(stemmer.stem(word) for word in text.split(' '))

    return text

In [ ]:
df['text_clean'] = df['text_clean'].apply(preprocess_data)
df.head()

,text,label,text_len,text_clean
0,Connect with Facebook for a personalized exper...,1,7,connect facebook person experi
1,Oops! Something went wrong. Please try again l...,1,8,oop someth went wrong plea tri later
2,"By clicking 'Accept,' you agree to our terms a...",1,10,click accept agr term condit
3,Great deals! Checkout now.,1,4,great deal checkout
4,Click here to manage your email preferences.,1,7,click manag email prefer


In [ ]:
x = df['text_clean']
y = df['label']

print(len(x), len(y))

2384 2384


In [ ]:
from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
# print(len(x_train), len(y_train))
# print(len(x_test), len(y_test))

# from sklearn.model_selection import train_test_split

# # Split the data into training and temporary sets (80% training, 20% temporary)
# X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)

# # Split the temporary set into validation and test sets (50% validation, 50% test)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Print the sizes of the resulting sets
# print("Training set size:", len(X_train))
# print("Validation set size:", len(X_val))
# print("Test set size:", len(X_test))


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# vect = CountVectorizer()
# vect.fit(x_train)

In [ ]:
# x_train_dtm = vect.transform(x_train)
# x_test_dtm = vect.transform(x_test)

In [ ]:
# vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)

In [ ]:
# from sklearn.feature_extraction.text import TfidfTransformer

# tfidf_transformer = TfidfTransformer()

# tfidf_transformer.fit(x_train_dtm)
# x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

# x_train_tfidf

In [ ]:
texts = df['text_clean']
target = df['label']

In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

2876

In [ ]:
def embed(corpus):
    return word_tokenizer.texts_to_sequences(corpus)

longest_train = max(texts, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))

train_padded_sentences = pad_sequences(
    embed(texts),
    length_long_sentence,
    padding='post'
)

train_padded_sentences

array([[ 758,  531,   83, ...,    0,    0,    0],
       [1180,  532,  759, ...,    0,    0,    0],
       [  63,  243,  139, ...,    0,    0,    0],
       ...,
       [   1,    2,    0, ...,    0,    0,    0],
       [ 283,  935,    0, ...,    0,    0,    0],
       [   1,    2,    0, ...,    0,    0,    0]], dtype=int32)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-02-10 10:33:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-02-10 10:33:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-02-10 10:33:16--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
!ls
!pwd

dark_dataset.csv   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
/content


In [ ]:
embedding_dim = 100
embeddings_dictionary = dict()

In [ ]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('/content/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_dictionary[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_dictionary))

Indexing word vectors.
Found 400000 word vectors.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
def generate_random_vector(dim):
    return np.random.rand(dim)

default_embedding_vector = generate_random_vector(embedding_dim)

embedding_matrix = np.zeros((vocab_length, embedding_dim))

# for word, index in word_tokenizer.word_index.items():
#     embedding_vector = embeddings_dictionary.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[index] = embedding_vector

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word, default_embedding_vector)
    embedding_matrix[index] = embedding_vector

embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.39392999, -0.038451  ,  0.22142   , ...,  0.28292   ,
         0.40625   , -0.2357    ],
       [ 0.86341   ,  0.69647998,  0.045794  , ..., -0.24698   ,
         0.17339   ,  0.94472998],
       ...,
       [-0.21602   , -0.50979   , -0.56703001, ..., -0.064713  ,
        -0.12592   , -0.23058   ],
       [ 0.19259163,  0.6235093 ,  0.01828754, ...,  0.94502364,
         0.35354801,  0.04439376],
       [ 0.19259163,  0.6235093 ,  0.01828754, ...,  0.94502364,
         0.35354801,  0.04439376]])

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     train_padded_sentences,
#     target,
#     testfrom sklearn.model_selection import train_test_split


X_train, X_temp, y_train, y_temp = train_test_split(train_padded_sentences,target , test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the sizes of the resulting sets
print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))
_size=0.20
# )



Training set size: 1907
Validation set size: 238
Test set size: 239


In [ ]:
def glove_lstm():
    model = Sequential()

    model.add(Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights = [embedding_matrix],
        input_length=length_long_sentence
    ))

    model.add(Bidirectional(LSTM(
        length_long_sentence,
        return_sequences = True,
        recurrent_dropout=0.2
    )))

    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = glove_lstm()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 74, 100)           287600    
                                                                 
 bidirectional (Bidirection  (None, 74, 148)           103600    
 al)                                                             
                                                                 
 global_max_pooling1d (Glob  (None, 148)               0         
 alMaxPooling1D)                                                 
                                                                 
 batch_normalization (Batch  (None, 148)               592       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 148)               0         
                                                        

In [ ]:
model = glove_lstm()

checkpoint = ModelCheckpoint(
    'model.h5',
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True
)
reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    verbose = 1,
    patience = 5,
    min_lr = 0.001
)
history = model.fit(
    X_train,
    y_train,
    epochs = 10,
    batch_size = 32,
    # validation_data = (X_test, y_test),
    validation_data=(X_val, y_val),
    verbose = 1,
    callbacks = [reduce_lr, checkpoint]
)

Epoch 1/10
60/60 [==============================] - ETA: 0s - loss: 0.7232 - accuracy: 0.6277
Epoch 1: val_loss improved from inf to 0.63884, saving model to model.h5
60/60 [==============================] - 27s 356ms/step - loss: 0.7232 - accuracy: 0.6277 - val_loss: 0.6388 - val_accuracy: 0.5798 - lr: 0.0010
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


60/60 [==============================] - ETA: 0s - loss: 0.5421 - accuracy: 0.7368
Epoch 2: val_loss improved from 0.63884 to 0.58111, saving model to model.h5
60/60 [==============================] - 17s 279ms/step - loss: 0.5421 - accuracy: 0.7368 - val_loss: 0.5811 - val_accuracy: 0.7941 - lr: 0.0010
Epoch 3/10
60/60 [==============================] - ETA: 0s - loss: 0.4443 - accuracy: 0.7923
Epoch 3: val_loss improved from 0.58111 to 0.51022, saving model to model.h5
60/60 [==============================] - 16s 273ms/step - loss: 0.4443 - accuracy: 0.7923 - val_loss: 0.5102 - val_accuracy: 0.7983 - lr: 0.0010
Epoch 4/10
60/60 [==============================] - ETA: 0s - loss: 0.3821 - accuracy: 0.8327
Epoch 4: val_loss improved from 0.51022 to 0.41026, saving model to model.h5
60/60 [==============================] - 21s 359ms/step - loss: 0.3821 - accuracy: 0.8327 - val_loss: 0.4103 - val_accuracy: 0.8824 - lr: 0.0010
Epoch 5/10
60/60 [==============================] - ETA: 0s - l

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [ ]:
y_preds = (model.predict(X_test) > 0.5).astype("int32")
confusion_matrix(y_test, y_preds)

8/8 [==============================] - 1s 71ms/step


array([[104,  10],
       [  7, 118]])

In [ ]:
accuracy_score(y_test, y_preds)

0.9288702928870293

In [ ]:
def preprocess_input(text):
    cleaned_text = clean_text(text)
    removed_stopwords = remove_stopwords(cleaned_text)
    stemmed_text = stemm_text(removed_stopwords)
    preprocessed_text = preprocess_data(stemmed_text)
    return preprocessed_text

def predict_dark_pattern_probability(text, model, tokenizer, max_length):
    preprocessed_text = preprocess_input(text)
    sequences = tokenizer.texts_to_sequences([preprocessed_text])
    padded_sequence = pad_sequences(sequences, maxlen=max_length, padding='post')

    probability = model.predict(padded_sequence)[0][0]

    return probability

text_to_predict = "Out of stock"
probability = predict_dark_pattern_probability(text_to_predict, model, word_tokenizer, length_long_sentence)

print(f"The probability of the text containing a dark pattern is: {probability}")


1/1 [==============================] - 0s 59ms/step
The probability of the text containing a dark pattern is: 0.9318353533744812
